In [1]:
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import pytorch3d.ops
from plyfile import PlyData, PlyElement
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from argparse import ArgumentParser, Namespace
import cv2
from tqdm import tqdm
from arguments import ModelParams, PipelineParams, ModelHiddenParams
from scene import Scene, GaussianModel
from gaussian_renderer import render, render_contrastive_feature, render_segmentation, render_mask
from segment_anything import (SamAutomaticMaskGenerator, SamPredictor, sam_model_registry)
from utils.sh_utils import SH2RGB
import imageio
from utils.segment_utils import *
from kmeans_pytorch import kmeans
from scene.hyper_loader import Load_hyper_data

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
sceneinfo = Load_hyper_data("./data/split-cookie/", 0.5, split ="train", object_masks=True)
views = [i for i in enumerate(sceneinfo)]
print(sceneinfo.num_classes)

tensor(215, dtype=torch.uint8)


In [12]:
sceneinfo[0].objects

tensor([[ 0,  0,  0,  ..., 31, 31, 31],
        [ 0,  0,  0,  ..., 31, 31, 31],
        [ 0,  0, 29,  ..., 31, 31, 31],
        ...,
        [41, 41, 41,  ..., 30, 30, 30],
        [41, 41, 41,  ..., 30, 30, 30],
        [41, 41, 41,  ..., 30, 30, 30]], dtype=torch.uint8)